In [23]:
import ConnectionConfig as cc
from delta import DeltaTable
from datetime import datetime

In [24]:
cc.setupEnvironment()
spark = cc.startLocalCluster("dimUser")
spark.getActiveSession()

In [25]:
run_timestamp =datetime.now()

In [26]:
dt_dimUserCurrent = DeltaTable.forPath(spark,"./spark-warehouse/user_dim_current")

In [27]:
dt_dimUserCurrent.toDF().createOrReplaceTempView("user_dim_current")

In [28]:
cc.set_connectionProfile("veloDB")

df_operational_users = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

df_operational_users.createOrReplaceTempView("operational_users")


In [29]:
df_user_dim_new = spark.sql( """
    SELECT uuid() as source_userSK,  
           userid as source_user_id,  
           name as source_name,  
           zipcode as source_zipcode,  
           city as source_city,  
           md5(concat(name, zipcode, city)) as source_md5  
    FROM operational_users
""")

df_user_dim_new.createOrReplaceTempView("user_dim_new")

# Debugging
df_user_dim_new.show()


+--------------------+--------------+--------------------+--------------+--------------------+--------------------+
|       source_userSK|source_user_id|         source_name|source_zipcode|         source_city|          source_md5|
+--------------------+--------------+--------------------+--------------+--------------------+--------------------+
|89844ca0-80db-48b...|             1|         Bouman Lars|          2060|           Antwerpen|5326bbf5b41d3f9dc...|
|0e1a4700-2ba8-4d3...|             2|   van der Zee Julia|          2610| Wilrijk (Antwerpen)|1491518a1ac33937d...|
|53a96556-56a1-47c...|             3|     de Boer Ricardo|          2160|           Wommelgem|2b6e4d3ed6e925255...|
|c685bed1-0b5a-422...|             4|      Willems Angela|          2900|             Schoten|49a5ddc033ae5f2b7...|
|b29baab0-50e0-44c...|             5|    Heijnen Patricia|          2540|                Hove|c87269ed29b9096f7...|
|8672aab5-1ce2-45d...|             6|      Driessen Anouk|          2650

In [30]:
detectedChanges = spark.sql("""
    SELECT 
        source.source_userSK,  
        source.source_user_id,  
        source.source_name,  
        source.source_zipcode,  
        source.source_city,  
        source.source_md5,  
        dwh.source_user_id AS dwh_user_id,  -- Rename to avoid duplicate column names
        dwh.source_md5 AS dwh_md5 
    FROM user_dim_new source  
    LEFT OUTER JOIN user_dim_current dwh  
    ON dwh.source_user_id = source.source_user_id AND dwh.current = true  
    WHERE dwh.source_user_id IS NULL OR dwh.source_md5 <> source.source_md5
""")

detectedChanges.createOrReplaceTempView("detectedChanges")


In [31]:
df_user_upserts = spark.sql(f"""
    SELECT 
        source_userSK as userSK,
        source_user_id as user_id,
        source_name as name,
        source_city as city,
        source_zipcode as zipcode,
        to_timestamp('{run_timestamp}') as scd_start,
        to_timestamp('2100-12-12', 'yyyy-MM-dd') as scd_end,
        source_md5 as md5,
        true as current
    FROM detectedChanges  
    UNION
    SELECT  
        userSK,
        source_user_id,
        source_name,
        source_city,
        source_zipcode,
        scd_start,
        to_timestamp('{run_timestamp}') as scd_end,
        source_md5, 
        false 
    FROM user_dim_current  --  Use user_dim_current instead of detectedChanges
    WHERE source_user_id IN (SELECT source_user_id FROM detectedChanges)  
    AND current = true  --  Ensure we only expire current records
""")


In [33]:
df_user_upserts.createOrReplaceTempView("upserts")

In [34]:

# DEBUG CODE TO SHOW CONTENT OF UPSERTS
spark.sql("select * from upserts").show()

+--------------------+-------+--------------------+--------------------+-------+--------------------+-------------------+--------------------+-------+
|              userSK|user_id|                name|                city|zipcode|           scd_start|            scd_end|                 md5|current|
+--------------------+-------+--------------------+--------------------+-------+--------------------+-------------------+--------------------+-------+
|8672aab5-1ce2-45d...|      6|      Driessen Anouk|              Edegem|   2650|2025-03-12 10:16:...|2100-12-12 00:00:00|ae81e29e2fb7849e7...|   true|
|f0d41950-e891-458...|     10|van der Steen Carmen|           Antwerpen|   2020|2025-03-12 10:16:...|2100-12-12 00:00:00|08edabd4bcab22ee8...|   true|
|dc27a13f-c985-422...|     11| de Ruijter Daniëlle|  Burcht/Zwijndrecht|   2070|2025-03-12 10:16:...|2100-12-12 00:00:00|56b0d91f98d932fcd...|   true|
|d7b9de23-c111-418...|     14|   van der Veer Romy|    Kontich/Waarloos|   2550|2025-03-12 10:

In [35]:

spark.sql("""
    MERGE INTO user_dim_current AS target
    USING upserts AS source
    ON target.source_user_id = source.userID AND target.current = TRUE

    WHEN MATCHED AND target.source_md5 <> source.source_md5 THEN 
        UPDATE SET 
            target.scd_end = source.scd_start, 
            target.current = FALSE

    WHEN NOT MATCHED THEN 
        INSERT (userSK, source_user_id, source_name, source_city, source_zipcode, scd_start, scd_end, source_md5, current) 
        VALUES (source.userSK, source.userID, source.name, source.city, source.zipcode, source.scd_start, source.scd_end, source.source_md5, source.current);
""")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `source`.`userID` cannot be resolved. Did you mean one of the following? [`city`, `current`, `md5`, `name`, `scd_end`, `scd_start`, `userSK`, `user_id`, `zipcode`].; line 13 pos 31

In [44]:
dt_dimUserCurrent.toDF().sort("source_user_id", "scd_start").show(100)

+--------------------+--------------+--------------------+--------------------+--------------+-------------------+-------------------+--------------------+-------+
|              userSK|source_user_id|         source_name|         source_city|source_zipcode|          scd_start|            scd_end|          source_md5|current|
+--------------------+--------------+--------------------+--------------------+--------------+-------------------+-------------------+--------------------+-------+
|73cac594-54c6-4c9...|             1|         Bouman Lars|           Antwerpen|          2060|1999-01-01 00:00:00|2100-12-12 00:00:00|976f30b5dd22c74bb...|   true|
|a2e2c171-d426-4f6...|             2|   van der Zee Julia| Wilrijk (Antwerpen)|          2610|1999-01-01 00:00:00|2100-12-12 00:00:00|c34f85e721c0459e3...|   true|
|0cf8b315-a89e-44b...|             3|     de Boer Ricardo|           Wommelgem|          2160|1999-01-01 00:00:00|2100-12-12 00:00:00|7b8896e6b68fdbcd6...|   true|
|f6b22ebe-5b5f-4

In [45]:
spark.stop()